<a href="https://www.kaggle.com/code/shivam1298/bitcoin-price-prediction-using-rnn?scriptVersionId=120759485" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1 style="color:red; text-align:center">Goal of this model is to predict Bitcoin prices with help of Bitcoin historical data.</h1>

**Dataset:**
1. There is 1 csv file, created from 5 different csv files. 
2. CSV files for select bitcoin exchanges for the time period of Jan 2017 to Dec 2020, with minute to minute updates of 
    * OHLC (Open, High, Low, Close)
    * Volume in BTC and indicated currency
    * Weighted bitcoin price (Volume USD).
3. Timestamps are in date variable.
4. Timestamps without any trades or activity have their data fields forward filled from the last valid time period. 
5. If a timestamp is missing, or if there are jumps, this may be because the exchange (or its API) was down, the exchange (or its API) did not exist, or some other unforseen technical error in data reporting or gathering.

<h3 style="color:brown">Importing Libraries</h3>

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy


# Data Transofrmation
from sklearn.preprocessing import MinMaxScaler


# Keras & TF libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
from tensorflow.keras import layers


# Time Series model libraries & validation libraries
from scipy import stats
from pylab import rcParams
import statsmodels.api as sm
from itertools import product
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-poster')

<h3 style="color:brown">Reading & Analysing Data</h3>

In [ ]:
df_bitcoin = pd.read_csv("/kaggle/input/bitcoin-price-2017-2021/BTC_2017_to_2021.csv")
df_bitcoin.shape

In [ ]:
df_bitcoin.head()

In [ ]:
df_bitcoin.info()

In [ ]:
df_bitcoin.describe().T

**Extracting Date from date column**

In [ ]:
df_bitcoin["date_only"] = df_bitcoin.date.apply(lambda x : x[:10])
df_bitcoin.head()

In [ ]:
Price = pd.DataFrame(df_bitcoin.groupby('date_only')['Volume USD'].mean())
Price.shape

In [ ]:
Price.head()

In [ ]:
Price.plot(figsize=(18,8))
plt.show()

In [ ]:
min(df_bitcoin.date), max(df_bitcoin.date)

<h2 style="color:red; text-align:center">Recursive Neural Network</h2>

<h3 style="color:brown">Train/Test Split</h3>

In [ ]:
# using prediction as 4 months, we're using 120 for train/test split

prediction_days = len(Price)-120

train = Price.iloc[:prediction_days]
test = Price.iloc[prediction_days:]

In [ ]:
train.shape, test.shape

<h3 style="color:brown">Data Preprocessing</h3>

In [ ]:
# Data preprocess

sc = MinMaxScaler()

sc.fit(train)

scaled_train = sc.transform(train)
scaled_test = sc.transform(test)

X_train = scaled_train[0:len(scaled_train)-1]
y_train = scaled_train[1:len(scaled_train)]

X_test = scaled_train[0:len(scaled_test)-1]
y_test = scaled_train[1:len(scaled_test)]

<h3 style="color:brown">RNN Model</h3>

**Model 1**

In [ ]:
%%time

regressor = Sequential()

# Adding the input layer and the LSTM layer
regressor.add(LSTM(units = 4, activation = 'relu', input_shape = (None, 1)))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 5, epochs = 50)

<h3 style="color:brown">Testing the Model</h3>

In [ ]:
predicted_BTC_price = regressor.predict(scaled_test)

predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)

In [ ]:
test["Predictions_Model1"] = predicted_BTC_price
test

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(test.reset_index()["date_only"], test.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price, color = 'blue', label = 'Predicted BTC Price')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

**Observations:** We can see similar predictions but it doesn't predict well towards the end. Let's try early stopping callback.

**Model 2**

In [ ]:
early_stop = EarlyStopping(monitor='loss',patience=2)

In [ ]:
%%time

regressor2 = Sequential()

# Adding the input layer and the LSTM layer
regressor2.add(LSTM(units = 128, activation = 'relu', input_shape = (None, 1)))

regressor2.add(Dense(units = 1))

regressor2.add(Dense(units = 1))

# Compiling the RNN
regressor2.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor2.fit(X_train, y_train, batch_size = 5, epochs = 50, callbacks=[early_stop])

In [ ]:
losses = pd.DataFrame(regressor2.history.history)
losses.plot()

<h3 style="color:brown">Testing the Model</h3>

In [ ]:
predicted_BTC_price2 = regressor2.predict(scaled_test)

predicted_BTC_price2 = sc.inverse_transform(predicted_BTC_price2)

In [ ]:
test["Predictions_Model2"] = predicted_BTC_price2
test

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(test.reset_index()["date_only"], test.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price2, color = 'green', label = 'Predicted BTC Price 2')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(test.reset_index()["date_only"], test.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price, color = 'blue', label = 'Predicted BTC Price 1')
plt.plot(predicted_BTC_price2, color = 'green', label = 'Predicted BTC Price 2')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

**Observations:** We can see similar predictions, more like both are overlapping each other, however model 1's predictions seems to be better than model 2's predictions. Let's try to update the train/test split to 60 days for more training data.

**Model 3**

<h3 style="color:brown">Train/Test Split</h3>

In [ ]:
# using prediction as 2 months, we're using 60 for train/test split

prediction_days = len(Price)-60

train_new = Price.iloc[:prediction_days]
test_new = Price.iloc[prediction_days:]

In [ ]:
train_new.shape, test_new.shape

<h3 style="color:brown">Data Preprocessing</h3>

In [ ]:
# Data preprocess

sc = MinMaxScaler()

sc.fit(train_new)

scaled_train_new = sc.transform(train_new)
scaled_test_new = sc.transform(test_new)

X_train_new = scaled_train_new[0:len(scaled_train_new)-1]
y_train_new = scaled_train_new[1:len(scaled_train_new)]

X_test_new = scaled_train[0:len(scaled_test_new)-1]
y_test_new = scaled_train[1:len(scaled_test_new)]

In [ ]:
early_stop = EarlyStopping(monitor='loss',patience=2)

In [ ]:
%%time

regressor3 = Sequential()

# Adding the input layer and the LSTM layer
regressor3.add(LSTM(units = 256, activation = 'relu', input_shape = (None, 1)))

# Adding the output layer
regressor3.add(Dense(units = 1))

# Compiling the RNN
regressor3.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor3.fit(X_train_new, y_train_new, batch_size = 5, epochs = 50, callbacks=[early_stop])

In [ ]:
losses = pd.DataFrame(regressor3.history.history)
losses.plot()

<h3 style="color:brown">Testing the Model</h3>

In [ ]:
predicted_BTC_price3 = regressor3.predict(scaled_test_new)

predicted_BTC_price3 = sc.inverse_transform(predicted_BTC_price3)

In [ ]:
test_new["Predictions_Model3"] = predicted_BTC_price3
test_new

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(test_new.reset_index()["date_only"], test_new.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price3, color = 'blue', label = 'Predicted BTC Price 3')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

In [ ]:
all_test_res = test.join(test_new["Predictions_Model3"], on="date_only", how="left")
all_test_res

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(all_test_res.reset_index()["date_only"], all_test_res.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(all_test_res.Predictions_Model1, color = 'blue', label = 'Predicted BTC Price 1')
plt.plot(all_test_res.Predictions_Model3, color = 'orange', label = 'Predicted BTC Price 3')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

**Observations:** By plotting the model 3 predictions with model 1's (which was better than model 2's predictions), we can observe that model 1 is quite similar to actual values throughout, but model 3 is better towards the end, which is understandable since the test data is for the end part of the whole 120 days.

<h3 style="color:brown">Trying Out Keras-Tuner for hyper-parameter tuning</h3>

In [ ]:
def model_build(hp):
    model = keras.Sequential()
    model.add(layers.LSTM(units = 256, activation = 'relu', input_shape = (None,1)))
    for i in range(hp.Int("num_layers", 1, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                                            activation='relu'))
    model.add(layers.Dense(1, activation='relu'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['mean_squared_error'])
    return model

**Initializing Tuner**

In [ ]:
tuner = RandomSearch(
    model_build,
    objective='val_mean_squared_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Bit_Coin_Price_Prediction')

In [ ]:
tuner.search_space_summary()

**Running Tuner**

In [ ]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

**Results from Tuner**

In [ ]:
tuner.results_summary()

**Model 4** : Using tuner's best results

In [ ]:
regressor4 = keras.Sequential()
regressor4.add(layers.LSTM(units = 256, activation = 'relu', input_shape = (None,1)))

regressor4.add(layers.Dense(units=512,activation='relu'))
regressor4.add(layers.Dense(units=256,activation='relu'))
regressor4.add(layers.Dense(units=512,activation='relu'))
regressor4.add(layers.Dense(units=416,activation='relu'))
regressor4.add(layers.Dense(units=192,activation='relu'))
regressor4.add(layers.Dense(units=32,activation='relu'))
regressor4.add(layers.Dense(units=128,activation='relu'))
regressor4.add(layers.Dense(units=512,activation='relu'))
regressor4.add(layers.Dense(units=128,activation='relu'))
regressor4.add(layers.Dense(units=416,activation='relu'))
regressor4.add(layers.Dense(units=512,activation='relu'))
regressor4.add(layers.Dense(units=96,activation='relu'))
regressor4.add(layers.Dense(units=416,activation='relu'))
regressor4.add(layers.Dense(units=512,activation='relu'))
regressor4.add(layers.Dense(units=96,activation='relu'))
regressor4.add(layers.Dense(units=416,activation='relu'))
regressor4.add(layers.Dense(units=288,activation='relu'))


regressor4.add(layers.Dense(1, activation='tanh'))
regressor4.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='mean_squared_error',
        metrics="mse")

In [ ]:
# Fitting the RNN to the Training set
regressor4.fit(X_train, y_train, batch_size = 7, epochs = 50, callbacks=[early_stop])

In [ ]:
losses = pd.DataFrame(regressor4.history.history)
losses.plot()

<h3 style="color:brown">Testing the Model</h3>

In [ ]:
predicted_BTC_price4 = regressor4.predict(scaled_test)

predicted_BTC_price4 = sc.inverse_transform(predicted_BTC_price4)

In [ ]:
test["Predictions_Model4"] = predicted_BTC_price4
test

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(test.reset_index()["date_only"], test.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price4, color = 'blue', label = 'Predicted BTC Price 4')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

**This is the best fit model till now**

In [ ]:
all_test_res["Predictions_Model4"] = test["Predictions_Model4"]
all_test_res

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(all_test_res.reset_index()["date_only"], all_test_res.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(all_test_res.Predictions_Model1, color = 'blue', label = 'Predicted BTC Price 1')
plt.plot(all_test_res.Predictions_Model4, color = 'orange', label = 'Predicted BTC Price 4')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

**Observations:**
We can see both the models are quite close, however model 4 is close to the actual values throughout. Although model 1's MSE is less than model 4, still in the plot we can see model 4 as the best model till now.

**Trying the second best result from keras-tuner**

**Model 5**

In [ ]:
regressor5 = keras.Sequential()
regressor5.add(layers.LSTM(units = 256, activation = 'relu', input_shape = (None,1)))

regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))
regressor5.add(layers.Dense(units=32,activation='relu'))


regressor5.add(layers.Dense(1, activation='tanh'))
regressor5.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.00001),
        loss='mean_squared_error',
        metrics=['mean_squared_error'])

In [ ]:
# Fitting the RNN to the Training set
regressor5.fit(X_train, y_train, batch_size = 7, epochs = 50, callbacks=[early_stop])

In [ ]:
losses = pd.DataFrame(regressor5.history.history)
losses.plot()

<h3 style="color:brown">Testing the Model</h3>

In [ ]:
predicted_BTC_price5 = regressor5.predict(scaled_test)

predicted_BTC_price5 = sc.inverse_transform(predicted_BTC_price5)

In [ ]:
test["Predictions_Model5"] = predicted_BTC_price5
test

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(test.reset_index()["date_only"], test.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price5, color = 'blue', label = 'Predicted BTC Price 5')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

In [ ]:
all_test_res["Predictions_Model5"] = test["Predictions_Model5"]
all_test_res

**Comparision with the best model till now**

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(all_test_res.reset_index()["date_only"], all_test_res.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(all_test_res.Predictions_Model4, color = 'blue', label = 'Predicted BTC Price 4')
plt.plot(all_test_res.Predictions_Model5, color = 'green', label = 'Predicted BTC Price 5')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

**Observations:**
We can see both the models are quite close, however model 4 is close to the actual values throughout. Although model 5's MSE is less than model 4, still in the plot we can see model 4 as the winning model.

**Model 6**

In [ ]:
regressor6 = keras.Sequential()
regressor6.add(layers.LSTM(units = 256, activation = 'relu', input_shape = (None,1)))

regressor6.add(layers.Dense(units=512,activation='relu'))
regressor6.add(layers.Dense(units=256,activation='relu'))
regressor6.add(layers.Dense(units=512,activation='relu'))
regressor6.add(layers.Dense(units=416,activation='relu'))
regressor6.add(layers.Dense(units=192,activation='relu'))
regressor6.add(layers.Dense(units=32,activation='relu'))
regressor6.add(layers.Dense(units=128,activation='relu'))
regressor6.add(layers.Dense(units=512,activation='relu'))
regressor6.add(layers.Dense(units=128,activation='relu'))
regressor6.add(layers.Dense(units=416,activation='relu'))
regressor6.add(layers.Dense(units=512,activation='relu'))
regressor6.add(layers.Dense(units=96,activation='relu'))
regressor6.add(layers.Dense(units=416,activation='relu'))
regressor6.add(layers.Dense(units=512,activation='relu'))
regressor6.add(layers.Dense(units=96,activation='relu'))
regressor6.add(layers.Dense(units=416,activation='relu'))
regressor6.add(layers.Dense(units=288,activation='relu'))


regressor6.add(layers.Dense(1, activation='tanh'))
regressor6.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.00001),
        loss='mean_squared_error',
        metrics="mse")

In [ ]:
# Fitting the RNN to the Training set
regressor6.fit(X_train, y_train, batch_size = 10, epochs = 50, callbacks=[early_stop])

In [ ]:
losses = pd.DataFrame(regressor6.history.history)
losses.plot()

<h3 style="color:brown">Testing the Model</h3>

In [ ]:
predicted_BTC_price6 = regressor6.predict(scaled_test)

predicted_BTC_price6 = sc.inverse_transform(predicted_BTC_price6)

In [ ]:
test["Predictions_Model6"] = predicted_BTC_price6
test

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(test.reset_index()["date_only"], test.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price6, color = 'blue', label = 'Predicted BTC Price 6')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

In [ ]:
all_test_res["Predictions_Model6"] = test["Predictions_Model6"]
all_test_res

**Comparision with the best model**

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
plt.plot(all_test_res.reset_index()["date_only"], all_test_res.reset_index()["Volume USD"], color = 'red', label = 'Real BTC Price')
plt.plot(all_test_res.Predictions_Model4, color = 'blue', label = 'Predicted BTC Price 4')
plt.plot(all_test_res.Predictions_Model6, color = 'green', label = 'Predicted BTC Price 6')
plt.title('BTC Price Prediction', fontsize=40)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 15})
plt.show()

<h3 style="color:brown">Best Model</h3>

After 6 iterations using different techniques/methods:
1. We can observe that mode 6 is the most close to the low values, whereas model 4 is close to all the high values.
2. However, model 4 is close throughout the timeline. 

Hence, `model 4 is the best model`.